In [6]:
import numpy as np
import pandas as pd
import networkx as nx
from models.dynotears_modified import from_pandas_dynamic
from metrics import edge_errors, edge_accurate, shd
from cdt.metrics import SID

i_dataset, shd_w,sid_w,fn_w,fp_w,rev_w, shd_a,sid_a,fn_a,fp_a,rev_a, shd_wa,sid_wa,fn_wa,fp_wa,rev_wa = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
results = [{'i_dataset':i_dataset,
            'shd_w':shd_w, 'sid_w': sid_w, 'fn_w':fn_w, 'fp_w': fp_w, 'rev_w': rev_w,
            'shd_a':shd_a, 'sid_a': sid_a, 'fn_a':fn_a, 'fp_a': fp_a, 'rev_a': rev_a,
            'shd_wa':shd_wa, 'sid_wa': sid_wa, 'fn_wa':fn_wa, 'fp_wa': fp_wa, 'rev_wa': rev_wa}] 
results = pd.DataFrame(results)
results.to_csv('10_p1_smp5500_dynotears_results.csv',header=True) #header=False


In [7]:
def save_results(i_dataset=None, w_est=None, a_est=None, true_dag=None):
    wa = np.hstack((np.vstack((w_est,a_est)),np.zeros((w_est.shape[0]+a_est.shape[0],a_est.shape[0]))))
    # if w_est == None:
    #     w_est = wa[:num_nodes,:num_nodes]
    #     a_est = wa[num_nodes:,:num_nodes]
    num_nodes = w_est.shape[0]
    fn_w, fp_w, rev_w = edge_errors(pred=w_est, target=true_dag[:num_nodes,:num_nodes])
    fn_a, fp_a, rev_a = edge_errors(pred=a_est, target=true_dag[num_nodes:,:num_nodes])
    fn_wa, fp_wa, rev_wa = edge_errors(wa, true_dag)
    shd_w = shd(pred=w_est, target=true_dag[:num_nodes,:num_nodes])
    shd_a = shd(pred=a_est, target=true_dag[num_nodes:,:num_nodes])
    shd_wa = shd(wa, true_dag)
    sid_w = SID(target=true_dag[:num_nodes,:num_nodes], pred=w_est)
    sid_a = SID(target=true_dag[num_nodes:,:num_nodes], pred=a_est)
    sid_wa = SID(true_dag, wa)
    results = [{'i_dataset':i_dataset,
                'shd_w':shd_w, 'sid_w': sid_w, 'fn_w':fn_w, 'fp_w': fp_w, 'rev_w': rev_w,
                'shd_a':shd_a, 'sid_a': sid_a, 'fn_a':fn_a, 'fp_a': fp_a, 'rev_a': rev_a,
                'shd_wa':shd_wa, 'sid_wa': sid_wa, 'fn_wa':fn_wa, 'fp_wa': fp_wa, 'rev_wa': rev_wa}] 
    results = pd.DataFrame(results)
    print(results)
    results.to_csv('10_p1_smp5500_dynotears_results.csv', mode='a', header=False) #header=False

In [8]:
load_path = '/home/lipeiwen.lpw/TECDI/data_simu/10_p1_smp5500/'
num_nodes = 10
for i_dataset in range(1,11):
    dag = np.load(load_path+f'DAG{i_dataset}.npy')
    df_g = pd.DataFrame(np.load(load_path+f'data{i_dataset}.npy')).iloc[:,:num_nodes]
    df_g = df_g.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
    g_learnt, w_est, a_est  = from_pandas_dynamic(df_g, 1, lambda_w=.1,lambda_a=.1,w_threshold=.5)
    w_est = np.where(w_est==0,w_est,1)
    a_est = np.where(a_est==0,a_est,1)
    save_results(i_dataset=i_dataset,w_est=w_est,a_est=a_est,true_dag=dag)

   i_dataset  shd_w sid_w  fn_w  fp_w  rev_w  shd_a sid_a  fn_a  fp_a  rev_a  \
0          1   10.0  28.0  10.0   0.0    0.0   16.0  57.0  15.0   1.0    0.0   

   shd_wa sid_wa  fn_wa  fp_wa  rev_wa  
0    26.0   67.0   25.0    1.0     0.0  
   i_dataset  shd_w sid_w  fn_w  fp_w  rev_w  shd_a sid_a  fn_a  fp_a  rev_a  \
0          2    9.0  24.0   8.0   1.0    0.0   30.0  90.0  29.0   1.0    0.0   

   shd_wa sid_wa  fn_wa  fp_wa  rev_wa  
0    39.0  132.0   37.0    2.0     0.0  
   i_dataset  shd_w sid_w  fn_w  fp_w  rev_w  shd_a sid_a  fn_a  fp_a  rev_a  \
0          3   18.0  57.0  18.0   0.0    0.0   24.0  70.0  24.0   0.0    0.0   

   shd_wa sid_wa  fn_wa  fp_wa  rev_wa  
0    42.0  117.0   42.0    0.0     0.0  
   i_dataset  shd_w sid_w  fn_w  fp_w  rev_w  shd_a sid_a  fn_a  fp_a  rev_a  \
0          4    8.0  16.0   7.0   0.0    1.0   32.0  88.0  30.0   2.0    0.0   

   shd_wa sid_wa  fn_wa  fp_wa  rev_wa  
0    40.0  131.0   37.0    2.0     1.0  
   i_dataset  shd_w sid_w  f

In [9]:
import statistics
res = pd.read_csv('10_p1_smp5500_dynotears_results.csv')
shd_wa = res['shd_wa'][1:]
shd_w = res['shd_w'][1:]
shd_a = res['shd_a'][1:]
sid_wa = res['sid_wa'][1:]
print('shd_wa:',round(np.mean(shd_wa),2),'\pm',round(statistics.stdev(shd_wa),2))
print('shd_w:',round(np.mean(shd_w),2),'\pm',round(statistics.stdev(shd_w),2))
print('shd_a:',round(np.mean(shd_a),2),'\pm',round(statistics.stdev(shd_a),2))
print('sid_wa:',round(np.mean(sid_wa),2),'\pm',round(statistics.stdev(sid_wa),2))

shd_wa: 36.0 \pm 5.21
shd_w: 11.8 \pm 3.36
shd_a: 24.2 \pm 4.92
sid_wa: 118.6 \pm 20.66
